# Wordle Auswertung

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import re
import seaborn as sns
import numpy as np
import datetime

df = pd.read_json("result.json")

nerdle = []
wordle = []
sechsmalfuenf = []
dungleon = []
for x in [d for d in df.messages]:
    date = datetime.datetime.strptime(x["date"],"%Y-%m-%dT%H:%M:%S")
    if x["type"] == "message":
        if type(x["text"]) == list and len(x["text"]) >= 2:
            if type(x["text"][0]) == dict:
                if x["text"][0]["text"] == "6mal5.com":
                    result = re.findall("[0-6X]/[0-6]", x["text"][1])
                    if result:
                        sechsmalfuenf.append({'name': x["from"], 'result': result[0][0], 'date': date})
                if x["text"][0]["text"] == "https://www.dungleon.com":
                    result = re.findall("[0-6X]/[0-6]", x["text"][1])
                    if result:
                        dungleon.append({'name': x["from"], 'result': result[0][0], 'date': date})
        else:
            if "nerdlegame" in x["text"]:
                result = re.findall("[0-6X]/[0-6]", x["text"])
                if result:
                    nerdle.append({'name': x["from"], 'result': int(result[0][0]), 'date': date})
            elif "Wordle" in x["text"]:
                result = re.findall("[0-6X]/[0-6]", x["text"])
                if result:
                    wordle.append({'name': x["from"], 'result': result[0][0], 'date': date})

sns.set(style='darkgrid')

In [ ]:
def plot_game(game_name, data):
    fig, axs = plt.subplots(2, figsize=(15,10))
    plt.suptitle("%s Ergebnisse" %game_name)    

    df = pd.DataFrame.from_dict(data)
    names = [name for name in np.sort(df["name"].unique())]
    results = [result for result in np.sort(df["result"].unique())]
    
    ## Absolute Frequency
    abs_freq = pd.DataFrame()
    abs_freq = df.value_counts(normalize=False).reset_index()
    abs_freq.columns = ["name", "result", "count"]

    ax = sns.barplot(ax=axs[0], x="name", y="count", hue="result", data=abs_freq, order=names, hue_order=results)
    ax.set(xlabel="Person", ylabel="Anzahl")
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    ax.yaxis.grid(False)
    ax.xaxis.grid(True)

    ## Relative Frequency
    df = pd.DataFrame.from_dict(data)
    rel_freq = pd.DataFrame()
    for name in df["name"].unique():
        temp = df.loc[df["name"] == name]
        temp = temp.value_counts(normalize=True).reset_index()
        temp.columns = ["name", "result", "count"]
        rel_freq = pd.concat([rel_freq, temp], ignore_index=True)
    rel_freq["count"] = 100 * rel_freq["count"]

    ax = sns.barplot(ax=axs[1], x="name", y="count", hue="result", data=rel_freq, order=names, hue_order=results)
    ax.set(xlabel="Person", ylabel="Prozent")
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    ax.yaxis.grid(False)
    ax.xaxis.grid(True)
    
    fig.savefig(game_name + ".png")

In [ ]:
def plot_weekday(data_list, lable_list):
    fig, axs = plt.subplots(1, figsize=(15,7))
    days = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday', 'Sunday']
    
    games_df = pd.DataFrame()
    min_date = None
    max_date = None
    for i, data in enumerate(data_list):
        df = pd.DataFrame.from_dict(data)        
        
        
        tmp_min = df["date"].min()
        tmp_max = df["date"].max()
        min_date = tmp_min if not min_date else min_date
        max_date = tmp_max if not max_date else max_date
        
        min_date = tmp_min if tmp_min < min_date else min_date
        max_date = tmp_max if tmp_max > max_date else max_date
                 
        week_df = df["result"].groupby(df['date'].dt.day_name()).count().reset_index()
        week_df["game"] = lable_list[i]
        week_df.columns = ["weekday", "result", "game"]
        
        games_df = pd.concat([games_df, week_df], ignore_index=True)
        
    ax = sns.barplot(x="weekday", y="result", hue="game", data=games_df, order=days)
    ax.set(ylabel="Anzahl an Worlde", xlabel="Wochentag")
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.suptitle("Ergebnisse nach Wochentag zwischen %s und %s" %(min_date.strftime("%b %Y"), max_date.strftime("%b %Y")))
    fig.savefig("wordles_weekdays.png")

In [ ]:
plot_game("Dungleon", dungleon)

In [ ]:
plot_game("Wordle", wordle)

In [ ]:
plot_game("6mal5.com", sechsmalfuenf)

In [ ]:
plot_game("Nerdle", nerdle)

In [ ]:
plot_weekday([wordle, sechsmalfuenf, nerdle, dungleon], ["Wordle", "6mal5.com", "Nerdle", "Dungleon"])